In [1]:
import pandas as pd
import prepare
from prepare import prep_equip_df
import explore

In [2]:
def get_hwyrail():
    df = pd.read_csv('hwy_rail_accidents_2012.csv', index_col=False)
    df1 = pd.read_csv('hwy_rail_accidents_2013.csv', index_col=False)  
    df2 = pd.read_csv('hwy_rail_accidents_2014.csv', index_col=False)  
    df3 = pd.read_csv('hwy_rail_accidents_2015.csv', index_col=False)  
    df4 = pd.read_csv('hwy_rail_accidents_2016.csv', index_col=False)  
    df5 = pd.read_csv('hwy_rail_accidents_2017.csv', index_col=False)  
    df6 = pd.read_csv('hwy_rail_accidents_2018.csv', index_col=False)  
    df7 = pd.read_csv('hwy_rail_accidents_2019.csv', index_col=False)  
    df8 = pd.read_csv('hwy_rail_accidents_2020.csv', index_col=False) 
    big_df = pd.concat([df, df1, df2, df3, df4, df5, df6, df7, df8])
    return big_df

In [3]:
hwy_df = get_hwyrail()
hwy_df

,AMTRAK,IYR,IMO,RAILROAD,INCDTNO,IYR2,IMO2,RR2,INCDTNO2,IYR3,...,NARR1,NARR2,NARR3,NARR4,NARR5,SUBDIV,ROADCOND,VIDEOT,VIDEOU,Unnamed: 103
0,NaN,12,3,UP,0312RS009,NaN,NaN,,,12.0,...,HIEHWAY USER'S ACTIONS: STOPPED ON CROSSING ( ...,NaN,NaN,NaN,NaN,WOODBRIDGE,B,1,2,NaN
1,NaN,12,3,UP,0312RS025,NaN,NaN,,,12.0,...,HIGHWAY USER'S ACTIONS: STOPPED ON CROSSING ( ...,D LEFT THE SCENE.,NaN,NaN,NaN,FRESNO SUB,A,1,2,NaN
2,NaN,12,10,BNSF,CA1012201,NaN,NaN,,,12.0,...,DRIVER AGE UNKNOW. 41: ABANDONED VEHICLE,NaN,NaN,NaN,NaN,SAN BERNARDINO,A,1,2,NaN
3,NaN,12,10,BNSF,CA1012202,NaN,NaN,,,12.0,...,12/18/12 PER CONTRA COSTA COUNTY CORONERS REPO...,NaN,NaN,NaN,NaN,STOCKTON,A,1,1,NaN
4,NaN,12,7,BNSF,CA0712201,NaN,NaN,,,12.0,...,NaN,NaN,NaN,NaN,NaN,STOCKTON,A,1,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,NaN,20,2,SFRV,2132020,NaN,NaN,,,20.0,...,TRAIN P645 WAS TRAVELLING SOUTHBOUND AND STRUC...,ALLY INJURED; AGE IS CURRENTLY UNKNOWN.,NaN,NaN,NaN,SOUTHFLRAILCORRIDOR,A,2,2,NaN
1887,NaN,20,1,SFRV,1052020,NaN,NaN,,,20.0,...,TWO MALES ON A BICYCLE RAN UNDER CROSSING GATE...,HE BIKE WAS HIT AND KILLED WHILE THE OTHER ONE...,PORT IS PENDING SO AGE OF DECEASED IS STILL UN...,NaN,NaN,SOUTHFLRAILCORRIDOR,A,1,1,NaN
1888,NaN,20,1,SFRV,1092020,NaN,NaN,,,20.0,...,TRAIN P647 STRUCK A VEHICLE THAT STOPPED ON TH...,CT. HE WAS TRAVELING EAST IN THE WESTBOUND TRA...,NaN,NaN,NaN,SOUTHFLRAILCORRIDOR,A,1,1,NaN
1889,NaN,20,1,SFRV,1172020,NaN,NaN,,,20.0,...,TRAIN P628 STRUCK A VEHICLE THAT STOPPED ON TH...,UNINJURED.,NaN,NaN,NaN,SOUTHFLRAILCORRIDOR,A,2,2,NaN


In [4]:
hwy_df.describe().T

,count,mean,std,min,25%,50%,75%,max
IYR,18995.0,15.995051,2.548730,12.0,14.0,16.0,18.0,20.0
IMO,18995.0,6.598052,3.531982,1.0,3.0,7.0,10.0,12.0
IYR2,19.0,17.947368,1.682382,14.0,17.0,18.0,19.5,20.0
IMO2,19.0,7.157895,4.003653,1.0,3.0,8.0,11.0,12.0
IYR3,18879.0,16.006038,2.550739,12.0,14.0,16.0,18.0,20.0
IMO3,18879.0,6.596801,3.531095,1.0,3.0,7.0,10.0,12.0
DUMMY1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CASINJRR,18995.0,0.417531,1.278679,0.0,0.0,0.0,1.0,87.0
YEAR,18995.0,15.995051,2.548730,12.0,14.0,16.0,18.0,20.0
MONTH,18995.0,6.598052,3.531982,1.0,3.0,7.0,10.0,12.0


In [5]:
hwy_df = prepare.prep_hwy_df(hwy_df)
hwy_df

,railroad_company,station,county,state,region,city,vehicle_speed,vehicle_type,vehicle_direction,position,...,driver_gender,train_occupants,user_killed,user_injured,rail_killed,rail_injured,train_pass_killed,train_pass_injured,road_condtions,date
incdtno,,,,,,,,,,,,,,,,,,,,,
0312RS009,UP,WOODBRIDGE,SAN JOAQUIN,6,7,LODI,0.0,A,4,2,...,1,3,0,0,0,0,0,0,B,2012-03-16 09:15:00
0312RS025,UP,ACAMPO,SAN JOAQUIN,6,7,,0.0,A,4,2,...,1,2,0,0,0,0,0,0,A,2012-03-30 21:35:00
CA1012201,BNSF,LOS NIETOS,LOS ANGELES,6,7,SANTA FE SPRINGS,0.0,A,1,2,...,1,3,0,0,0,0,0,0,A,2012-10-07 20:25:00
CA0712201,BNSF,FRESNO,FRESNO,6,7,FRESNO,0.0,A,1,2,...,2,2,0,1,0,0,0,0,A,2012-07-28 20:55:00
0612RS011,UP,BAKERSFIELD,KERN,6,7,,50.0,C,4,3,...,1,3,0,0,0,0,0,0,A,2012-06-18 15:58:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137378,NS,BALTIMORE,BALTIMORE,24,2,BALTIMORE,2.0,A,1,3,...,1,2,0,0,0,0,0,0,A,2020-03-04 06:32:00
189236,CSX,INWOOD ST,PRINCE GEORGE'S,24,2,HYATTSVILLE,10.0,C,4,3,...,1,2,0,0,0,0,0,0,A,2020-03-21 10:15:00
188716,CSX,MONTEVIDEO ROAD,HOWARD,24,2,ELKRIDGE,15.0,J,4,3,...,1,3,0,0,0,0,0,0,A,2020-02-27 09:30:00


In [6]:
hwy_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12181 entries, 0312RS009 to 193825
Data columns (total 48 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   railroad_company    12181 non-null  object        
 1   station             12181 non-null  object        
 2   county              12181 non-null  object        
 3   state               12181 non-null  int64         
 4   region              12181 non-null  int64         
 5   city                12181 non-null  object        
 6   vehicle_speed       12181 non-null  float64       
 7   vehicle_type        12181 non-null  object        
 8   vehicle_direction   12181 non-null  object        
 9   position            12181 non-null  object        
 10  accident_type       12181 non-null  int64         
 11  hazmat_entity       12181 non-null  object        
 12  temp                12181 non-null  int64         
 13  visibility          12181 non-null  int64 

In [7]:
def get_equiprail():
    
    '''This function acquires the equipment rail accident data, concats all of the dataframes into one'''
    
    
    df = pd.read_csv('rail_equip_accidents_2012.csv', index_col=False)
    df1 = pd.read_csv('rail_equip_accidents_2013.csv', index_col=False)  
    df2 = pd.read_csv('rail_equip_accidents_2014.csv', index_col=False)  
    df3 = pd.read_csv('rail_equip_accidents_2015.csv', index_col=False)  
    df4 = pd.read_csv('rail_equip_accidents_2016.csv', index_col=False)  
    df5 = pd.read_csv('rail_equip_accidents_2017.csv', index_col=False)  
    df6 = pd.read_csv('rail_equip_accidents_2018.csv', index_col=False)  
    df7 = pd.read_csv('rail_equip_accidents_2019.csv', index_col=False)  
    df8 = pd.read_csv('rail_equip_accidents_2020.csv', index_col=False) 
    
    equip_df = pd.concat([df, df1, df2, df3, df4, df5, df6, df7, df8])
    
    return equip_df

In [8]:
equip_df = get_equiprail()

In [9]:
equip_df

,IYR,IMO,RAILROAD,INCDTNO,IYR2,IMO2,RR2,INCDTNO2,IYR3,IMO3,...,RCL,Latitude,Longitud,SIGNAL,MOPERA,ADJUNCT1,ADJUNCT2,ADJUNCT3,SUBDIV,Unnamed: 145
0,12,5,BNSF,GC0512102,NaN,NaN,NaN,NaN,12,5,...,0.0,29.755900,-95.293100,1,1,G,NaN,NaN,NORTH SHORE,NaN
1,12,5,BNSF,GC0512105,NaN,NaN,NaN,NaN,12,5,...,0.0,29.500571,-95.597089,2,5,Z,NaN,NaN,GALVESTON,NaN
2,12,2,NS,98074,12.0,2.0,CSX,100906,12,2,...,NaN,34.612981,-86.988133,1,1,B,NaN,NaN,ALABAMA,NaN
3,12,5,BNSF,GC0512106,NaN,NaN,NaN,NaN,12,5,...,0.0,31.446576,-96.207606,2,5,Z,NaN,NaN,HOUSTON,NaN
4,12,5,BNSF,GC0512112,NaN,NaN,NaN,NaN,12,5,...,0.0,29.772155,-95.384013,2,5,Z,NaN,NaN,HOUSTON,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135,20,2,NS,137017,NaN,NaN,NaN,NaN,20,2,...,0.0,37.271098,-79.997944,2,5,K,NaN,NaN,POCAHONTAS,NaN
2136,20,5,BNSF,PR0520107,NaN,NaN,NaN,NaN,20,5,...,0.0,44.216404,-105.293504,1,1,Q,NaN,NaN,ORIN,NaN
2137,20,5,NECR,NEC420520D,NaN,NaN,NaN,NaN,20,5,...,0.0,44.481200,-73.101600,2,2,NaN,NaN,NaN,ROXBURY,NaN
2138,20,1,CSX,188190,NaN,NaN,NaN,NaN,20,1,...,3.0,31.213524,-82.353588,2,5,NaN,NaN,NaN,THOMASVILLE,NaN


In [10]:
equip_df = prep_equip_df(equip_df)
equip_df

,railroad_company,accident_type,state,temp,visibility,weather,train_speed,train_direction,train_weight,train_type,...,total_damage,engineers_onduty,conductors_onduty,brakemen_onduty,region,typrr,lat,long,signal_type,date
incdtno,,,,,,,,,,,,,,,,,,,,,
GC0512102,BNSF,1,48,84,2,1,10,2.0,9811,1,...,25500,1.0,1.0,0.0,5,1,29.7559,-95.2931,1,2012-05-05 08:00:00
GC0512105,BNSF,1,48,70,4,3,5,2.0,17157,1,...,10940,1.0,1.0,0.0,5,1,29.500571,-95.597089,2,2012-05-11 02:25:00
GC0512106,BNSF,1,48,84,2,1,3,2.0,17304,1,...,10500,1.0,1.0,0.0,5,1,31.446576,-96.207606,2,2012-05-16 15:40:00
GC0512112,BNSF,1,48,70,4,1,5,3.0,14212,1,...,11664,1.0,1.0,0.0,5,1,29.772155,-95.384013,2,2012-05-31 03:36:00
GC0912118,BNSF,1,48,78,2,3,8,1.0,5918,1,...,255000,1.0,1.0,0.0,5,1,29.606297,-95.300054,2,2012-09-29 14:10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194579,CSX,9,51,38,2,3,3,3.0,0,7,...,12500,1.0,1.0,0.0,2,1,37.30244,-77.284923,2,2020-12-20 09:45:00
140029,NS,1,51,72,4,2,6,3.0,14242,1,...,175547,1.0,1.0,0.0,2,1,37.273201,-79.939819,1,2020-12-16 02:16:00
137017,NS,1,51,39,4,2,6,3.0,0,7,...,24683,1.0,1.0,0.0,2,1,37.271098,-79.997944,2,2020-02-01 23:20:00


In [11]:
equip_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11504 entries, GC0512102 to 162814
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   railroad_company   11504 non-null  object        
 1   accident_type      11504 non-null  int64         
 2   state              11504 non-null  int64         
 3   temp               11504 non-null  int64         
 4   visibility         11504 non-null  int64         
 5   weather            11504 non-null  int64         
 6   train_speed        11504 non-null  int64         
 7   train_direction    11504 non-null  float64       
 8   train_weight       11504 non-null  int64         
 9   train_type         11504 non-null  object        
 10  track_type         11504 non-null  int64         
 11  front_engines      11504 non-null  int64         
 12  loadfrght_cars     11504 non-null  int64         
 13  loadpass_cars      11504 non-null  int64         
 14  em

In [12]:
equip_df['railroad_company'].nunique()

5

In [13]:
train, validate, test = explore.train_validate_test_split(equip_df, 'railroad_company', seed=123)

In [14]:
train.head()

,railroad_company,accident_type,state,temp,visibility,weather,train_speed,train_direction,train_weight,train_type,...,total_damage,engineers_onduty,conductors_onduty,brakemen_onduty,region,typrr,lat,long,signal_type,date
incdtno,,,,,,,,,,,,,,,,,,,,,
124919,NS,1,36,48,4,2,2,4.0,0,7,...,21758,1.0,1.0,0.0,1,1.0,42.882771,-78.819282,1,2017-05-03 00:55:00
128409,NS,1,29,8,4,6,10,4.0,0,1,...,22677,1.0,1.0,0.0,6,1.0,39.421994,-92.437622,2,2018-02-06 00:45:00
SW0613108,BNSF,1,35,65,4,1,7,3.0,6893,1,...,488630,1.0,1.0,0.0,5,1,34.367942,-106.857886,2,2013-06-22 02:30:00
0817TC001,UP,3,19,65,4,1,4,3.0,5369,1,...,16357,1.0,1.0,0.0,6,1.0,42.028053,-93.57995,1,2017-08-01 01:22:00
0116LV004,UP,7,22,47,4,1,49,2.0,6053,1,...,59797,1.0,1.0,0.0,5,1,30.787575,-92.677632,1,2016-01-09 21:19:00


In [15]:
train.count(), validate.count(), test.count()

(railroad_company     6442
 accident_type        6442
 state                6442
 temp                 6442
 visibility           6442
 weather              6442
 train_speed          6442
 train_direction      6442
 train_weight         6442
 train_type           6442
 track_type           6442
 front_engines        6442
 loadfrght_cars       6442
 loadpass_cars        6442
 emptyfrght_cars      6442
 emptypass_cars       6442
 equip_damage         6442
 track_damage         6442
 cause                6442
 total_killed         6442
 total_injured        6442
 max_speed            6442
 total_damage         6442
 engineers_onduty     6442
 conductors_onduty    6442
 brakemen_onduty      6442
 region               6442
 typrr                6442
 lat                  6442
 long                 6442
 signal_type          6442
 date                 6442
 dtype: int64, railroad_company     2761
 accident_type        2761
 state                2761
 temp                 2761
 visibility   

In [16]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
accident_type,6442.0,3.542844,4.210843,1.0,1.0,1.0,7.00,13.0
state,6442.0,29.592207,16.211696,1.0,17.0,30.0,47.00,56.0
temp,6442.0,58.856877,22.181626,-36.0,42.0,61.0,75.00,110.0
visibility,6442.0,2.723844,1.061947,1.0,2.0,2.0,4.00,4.0
weather,6442.0,1.521267,0.943324,1.0,1.0,1.0,2.00,6.0
train_speed,6442.0,15.591897,20.174284,0.0,4.0,7.0,19.75,545.0
train_direction,6442.0,2.669047,1.102312,1.0,2.0,3.0,4.00,4.0
train_weight,6442.0,4239.179603,5282.094969,0.0,0.0,2386.0,6629.50,84414.0
track_type,6442.0,1.844924,0.849676,1.0,1.0,2.0,2.00,4.0
front_engines,6442.0,2.205681,1.177367,0.0,2.0,2.0,3.00,9.0
